In [18]:
import pandas as pd
import json
import os
from selectolax.parser import HTMLParser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from datetime import date

In [3]:
input_24 = pd.read_excel('Output/2024-05-27_output_filtrato_v1.xlsx')

In [6]:
input_24= input_24[input_24['stato']=='IN ATTESA DI PAGAMENTO']
#  input_24= input_24[input_24['compila']=='si']

In [14]:
def apply_mapping(value):
    if 'DECRETO INGIUNTIVO' in value:
        return 'DI - DECRETO INGIUNTIVO'
    elif 'ESECUZIONE MOBILIARE' in value:
        return 'EM - ESECUZIONE MOBILIARE'
    elif 'ORDINANZA' in value:
        return 'OR - ORDINANZA'
    elif 'ATTO VARIO' in value:
        return 'AV - ATTO VARIO'
    else:
        return None  # Default value if no match is found

# Apply the mapping to create the 'natura' column
input_24['natura'] = input_24['atto'].apply(apply_mapping)

C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_2584\3067407216.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_24['natura'] = input_24['atto'].apply(apply_mapping)


In [16]:
def extract_numero(value):
    numero_match = re.search(r'n\. (\d+)', value)
    if numero_match:
        return numero_match.group(1)
    else:
        return None

def extract_anno(value):
    anno_match = re.search(r'del (\d+)', value)
    if anno_match:
        return anno_match.group(1)
    else:
        return None

# Apply the functions to create the 'numero' and 'anno' columns
input_24['numero'] = input_24['atto'].apply(extract_numero)
input_24['anno'] = input_24['atto'].apply(extract_anno)

C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_2584\2239192135.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_24['numero'] = input_24['atto'].apply(extract_numero)
C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_2584\2239192135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_24['anno'] = input_24['atto'].apply(extract_anno)


In [17]:
input_24

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,natura,numero,anno
7,TBCO0012388330.074,LANZO ANTONIO FERNANDO,TARANTO UT DPTA,DECRETO INGIUNTIVO n. 07 del 2021,GIUDICE DI PACE DI S. GIORGIO,65/23 - 2023,TIBERIUS SPV S.R.L.,LANZO,IN ATTESA DI PAGAMENTO,di,07,2021
10,TBCO0012956551.074,FACCIO DIEGO,VICENZA UT DPVI,ESECUZIONE MOBILIARE n. 1162 del 2023,TRIBUNALE DI VICENZA,48 - 2024,TIBERIUS SPV S.R.L.,FACCIO,IN ATTESA DI PAGAMENTO,em,1162,2023
13,TBS2037576,SCIVOLETTO ANTONINA,VERONA 2 UT DPVR,ESECUZIONE MOBILIARE n. 2099 del 2023,TRIBUNALE DI VERONA,211 - 2024,TIBERIUS SPV S.R.L.,SCIVOLETTO,IN ATTESA DI PAGAMENTO,em,2099,2023
14,TBS24010855,WANG PENGFEI,TORINO 1 UT DPTO1,ESECUZIONE MOBILIARE n. 4529 del 2023,TRIBUNALE TORINO,1001 - 2024,TIBERIUS SPV S.R.L.,WANG,IN ATTESA DI PAGAMENTO,em,4529,2023
15,TBS24682111,KOUAKEB SELLAMI,BOLOGNA 2 UT DPBO,ESECUZIONE MOBILIARE n. 959 del 2023,TRIBUNALE,380/24 - 2024,TIBERIUS SPV S.R.L.,KOUAKEB SELLAMI,IN ATTESA DI PAGAMENTO,em,959,2023
17,TBS27898708,HAPAN PEDIGE PRAGEETH TUSITHA VIJAYALATH,VERONA 2 UT DPVR,ESECUZIONE MOBILIARE n. 373 del 2024,TRIBUNALE DI VERONA,534/24 - 2024,TIBERIUS SPV S.R.L.,HAPAN PEDIGE,IN ATTESA DI PAGAMENTO,em,373,2024
22,TBS52364746,PEPKOLAJ MIRJETA,GENOVA 1 UT DPGE,ESECUZIONE MOBILIARE n. 2805 del 2023,TRIBUNALE DI GENOVA,B640 - 2024,TIBERIUS SPV S.R.L.,PEPKOLAJ,IN ATTESA DI PAGAMENTO,em,2805,2023
23,TBS52495632,MD MOSTAFA,CREMONA UT DPCR,ESECUZIONE MOBILIARE n. 482 del 2023,TRIBUNALE DI CREMONA,519 - 2023,TIBERIUS SPV S.R.L.,MD,IN ATTESA DI PAGAMENTO,em,482,2023
24,TBS53001679,KALUGAMAGE ANTON SUJITH FERNANDO,TREVISO UT DPTV,DECRETO INGIUNTIVO n. 2830 del 2022,TRIBUNALE TREVISO,3852/23 - 2022,BLADE MANAGEMENT S.R.L.,KALUGAMAGE ANTON SUJITH FERNAN,IN ATTESA DI PAGAMENTO,di,2830,2022
27,TBS63943471,SCOGNAMIGLIO GIUSEPPE,NAPOLI 3 UT DPNA2,DECRETO INGIUNTIVO n. 158 del 2021,TRIBUNALE DI NAPOLI,4285 - 2021,BLADE MANAGEMENT S.R.L.,SCOGNAMIGLIO,IN ATTESA DI PAGAMENTO,di,158,2021


In [32]:
def click_button(driver, button):
    try:
        button.click()
    except Exception as e:
        # If clicking using regular method fails, try clicking using JavaScript
        driver.execute_script("arguments[0].click();", button)

def save_dict_to_json(dictionary, path):
    with open(path, 'w') as file:
        json.dump(dictionary, file)

In [42]:

def select_option(driver, dropdown_locator, option_text):
    try:
        dropdown_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(dropdown_locator))
        dropdown_element.click()
        
        option_locator = (By.XPATH, f"//option[contains(text(), '{option_text}')]")
        option_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(option_locator))
        option_element.click()
    except Exception as e:
        print(f"Error: {e}")
        raise
       

def compila(driver,office_name,ente,anno,natura,numero):
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    driver.get(url)

    try:
        #  inserire l'ufficio
        select_option(driver, (By.CSS_SELECTOR, "select#ufficio.form-control"), office_name)
        #clicca avanti
        select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
        click_button(driver, select_element)
        time.sleep(1)

        try:
            # clicca per inserire l'ente
            select_option(driver, (By.CSS_SELECTOR, "select#ente.form-control"), ente)
            # inserire l'anno
            select_option(driver, (By.CSS_SELECTOR, "select#anno.form-control"), anno)
            # inserire la natura
            select_option(driver, (By.CSS_SELECTOR, "select#natura.form-control"), natura)


            dropdown_element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#natura.form-control")))
            driver.execute_script("arguments[0].blur();", dropdown_element )
            
            # clicca per inserire il numero
            select_locator = (By.CSS_SELECTOR, "input#numero.form-control")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, numero)
                   
            time.sleep(2)

            #clicca avanti
            select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            click_button(driver, select_element)
            time.sleep(2)

            compila_f24_locator = (By.XPATH, "//a[contains(text(), 'Compila F24')]")
            compila_f24_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(compila_f24_locator))
            compila_f24_element.click()

            time.sleep(10)

        except Exception as e:
            print(f"Error: {e}")
            return []   

    except Exception as e:
        print(f"Error: {e}")
        return []

 
def main():
    options = webdriver.EdgeOptions()
    #options.add_argument("--headless")
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    
    compila(driver,office_name='TARANTO  UT DPTA',ente='GIUDICE DI PACE DI S. GIORGIO',anno='2021',natura='DI - DECRETO INGIUNTIVO',numero='07')

    driver.quit()

 
if __name__ == "__main__":
    main()
